In [2]:
import numpy as np
from numpy_ringbuffer import RingBuffer
import redis, time, signal, sys
import os
import json
from sklearn.cluster import KMeans

In [3]:
def str_to_vec(s):
    return np.array([float(x) for x in s[1:-1].split(' ')])

def key_to_vec(r, key_name):
    return str_to_vec(r.get(key_name))

def vec_to_str(v):
    return '[' + ','.join(x for x in v) + ']'
def signal_handler(signal, frame):
    global runloop
    runloop = False
    print(' ... Exiting data logger')

In [10]:
def predict(x, y, z, r, x_cross = -0.665):
    #     pz = z + 4.9 * x * x
    coeffz = np.polyfit(x, z, 2)
    coeffy = np.polyfit(x, y, 1)
    z_cross = coeffz[2] + coeffz[1] * x_cross + coeffz[0] * x_cross *x_cross
    y_cross = coeffy[1] + coeffy[0] * x_cross

    y_cross -= -1.138
    z_cross -= 0.2
    # r.set("sai2::FrankaPanda::control::target_position", vec_to_str([str(x_cross), str(y_cross), str(z_cross)]))
    #     return np.array([x_cross, y_cross, z_cross])
    return y_cross, z_cross


In [49]:
buffersize = 10
minpts = 2

xbuff = RingBuffer(capacity=buffersize)
ybuff = RingBuffer(capacity=buffersize)
zbuff = RingBuffer(capacity=buffersize)

ptcount = 0
if True:
    ptcount = min(ptcount + 1, buffersize)
#     xbuff.append()
    if ptcount > minpts:
        xp = np.array(xbuff[:ptcount])
        yp = np.array(ybuff[:ptcount])
        zp = np.array(zbuff[:ptcount])
        predict(xp, yp, zp, redis)
        
else:
    ptcount = max(ptcount - 1, 0)

In [45]:
xm = np.array([0, 1.1, 1.9, 3.2, 4])
ts = np.array([0, 1.01, 2, 3, 4])
# dx = np.diff(x)
rx = RingBuffer(capacity = 4, dtype=np.double)
rt = RingBuffer(capacity = 4, dtype=np.double)
for x in xm:
    rx.append(x)
for t in ts:
    rt.append(t)
    
print(rx)

<RingBuffer of array([1.1, 1.9, 3.2, 4. ])>


In [110]:
xm.shape

(5,)

In [158]:
OPTI_TIMESTAMP_KEY = "sai2::optitrack::timestamp"
OPTI_POS_KEY = "sai2::optitrack::pos_single_markers"

r_server = redis.StrictRedis(host='localhost', port=6379, db=0)

num_fxpts = 0
num_vals = 1000
fpts = np.empty((num_vals, 3))

prev_time = 0
ffidx = 0

while ffidx < num_vals:
    opti_time = json.loads(r_server.get(OPTI_TIMESTAMP_KEY).decode("utf-8"))
    poslist = r_server.get(OPTI_POS_KEY).decode("utf-8").split(";")
    
    if opti_time != prev_time:
        opti_pos = json.loads("["+",".join(("".join(poslist)).split(" "))+"]")
        
        prev_time = opti_time
        num_fxpts = max(num_fxpts, len(poslist))
        
        for i in range(len(poslist)):
            fpts[ffidx,:] = opti_pos[3*i:3*(i+1)]
            ffidx += 1
            if ffidx == num_vals:
                break
            
# fpts = fpts.reshape(ffidx,3)
    
kmeans = KMeans(n_clusters=num_fxpts, random_state=0).fit(fpts)
fcords = kmeans.cluster_centers_

print("fixed markers found")



fixed markers found


In [160]:
signal.signal(signal.SIGINT, signal_handler)
runloop = True
# count = 0

folder = 'opti'
if not os.path.exists(folder):
    os.makedirs(folder)

# date and time
timestamp = time.strftime("%x").replace('/','-') + '_' + time.strftime("%X").replace(':','-')

# file names
name = "cd_f10"
# name = "comp_kin_kp100_fc10"
# name = 'k50_comp_kin_fc5'

# open files
file = open(folder + '/' + name + '_' + timestamp,'w')
    
while(runloop):
    opti_time = json.loads(r_server.get(OPTI_TIMESTAMP_KEY).decode("utf-8"))
    poslist = r_server.get(OPTI_POS_KEY).decode("utf-8").split(";")
    
    if opti_time != prev_time:
        opti_pos = json.loads("["+",".join(("".join(poslist)).split(" "))+"]")
        
        for i in range(len(poslist)):
            pos = np.array(opti_pos[3*i:3*(i+1)]).reshape(1,3)
            dist = np.linalg.norm((pos - fcords), axis = 1)
            if np.all(dist > 0.1):
                print(pos)


[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 0.929124 -0

[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976259 -1.079369 -2.743068]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976259 -1.079369 -2.743068]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929124 -0.54856  -1.072433]]
[[ 1.976194 -1.078992 -2.74324 ]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 0.929115 -0

[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0

[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.976082 -1.078812 -2.743102]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929115 -0.548448 -1.072505]]
[[ 1.975795 -1

[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929057 -0.548514 -1.072401]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0

[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0

[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1.078903 -2.742562]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 1.975795 -1

[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0.548533 -1.07243 ]]
[[ 0.929074 -0

In [140]:
[[ 0.623062 -0.721789 -0.540295]]
[[ 0.623062 -0.721789 -0.540295]]
[[ 0.623062 -0.721789 -0.540295]]
[[ 0.623062 -0.721789 -0.540295]]
[[ 0.623068 -0.721773 -0.540314]]
[[ 0.623068 -0.721773 -0.540314]]
[[ 0.623068 -0.721773 -0.540314]]
[[ 0.623068 -0.721773 -0.540314]]
[[ 0.623068 -0.721773 -0.540314]]
[[ 0.623068 -0.721773 -0.540314]]



array([[ 0.76475558, -1.39660123, -0.13583848],
       [ 1.9484642 , -1.11358273, -2.80404456],
       [ 0.95339527, -0.37737727, -0.91167677],
       [ 0.95338377, -1.2960896 , -0.45333186],
       [ 0.69487697, -1.29385529, -0.37603391],
       [ 1.02487883, -1.39849526, -0.21458354]])

In [159]:
fcords

array([[ 2.06736783, -1.09452638, -2.88385868],
       [ 0.69516001, -1.29370887, -0.3759126 ],
       [ 1.02484512, -1.39841353, -0.21402795],
       [ 0.76505282, -1.39675571, -0.13596366],
       [ 0.9532663 , -1.29595934, -0.45327118]])

In [24]:
file = np.loadtxt("opti/ball_traj_06-05-19_12-53-59")

In [25]:
t, x, y, z = file.T

In [29]:
ptcount = 0
minpts = 9
buffersize = 10

buff = RingBuffer(capacity=buffersize, dtype=(np.float64, 4))

N = len(t)
for i in range(N):
    pos = x[i], y[i], z[i]
    
    if pos[2] > 0.2 and pos[0] < 1.4 and pos[0] > -0.665 and pos[1] < -0.638 and pos[1] > -1.638:
        buff.append(np.array([pos[0], pos[1], pos[2], t[i]]).reshape(1, 4))

    ptcount = min(ptcount + 1, buffersize)
    if ptcount > minpts:
        xp = np.array(buff[:ptcount, 0])
        yp = np.array(buff[:ptcount, 1])
        zp = np.array(buff[:ptcount, 2])
        yc, zc = predict(xp, yp, zp, None)
        print(i, yc, zc)

9 -0.15393027352897581 1.0143398621635058
10 -0.1544400222016611 0.9781679472593343
11 -0.1547806741977984 0.9481304309250418
12 -0.15611930512525474 0.9423554697187393
13 -0.15596013160083988 0.954736123738577
14 -0.1560204099572169 0.9815672983165642
15 -0.1552624762805359 0.9696416256611544
16 -0.15514421851358273 0.9428921800221497
17 -0.1545675941909126 0.9201322841327944
18 -0.15447909655415515 0.8959357518898687
19 -0.15479340701044264 0.8778918188763096
20 -0.15554964161142015 0.8790221670924865
21 -0.15571095989035055 0.8862674140729485
22 -0.15672758793745034 0.9006964873093526
23 -0.15684821339350807 0.8999504027710434
24 -0.15552158858085496 0.8932247351231002
25 -0.15600474226491823 0.882661615268624
26 -0.15660834646060962 0.8803216199947297
27 -0.1565913721339034 0.8695500438115198
28 -0.1568320653838653 0.8790129586765398
29 -0.15736604123665066 0.8802224412846014
30 -0.15869140675349303 0.8844929702954392
31 -0.15911647252806937 0.8823841653888935
32 -0.161953370594859